In [50]:
import torch
from transformers import BertTokenizer, BertModel, GPT2Tokenizer, GPT2Model
import pandas as pd
from tqdm import tqdm

from sklearn.model_selection import train_test_split
import pandas as pd
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import datetime

In [51]:


# Load and preprocess data
def load_data(file_path):
    data = pd.read_csv(file_path)
    texts = data['text'].tolist()
    scores = data['score'].tolist()
    return texts, scores

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [52]:
# Define the MLP model
class MLPModel(nn.Module):
    def __init__(self):
        super(MLPModel, self).__init__()
        self.l1 = nn.Sequential(
            
            # BERT 10
            nn.Linear(9984, 4096),
            # nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(4096, 1024),
            # nn.ReLU(),
            # nn.Linear(2048, 1024),
            # nn.ReLU(),
            nn.Linear(1024, 256),
            # nn.Dropout(0.2),
            nn.ReLU(),
            # nn.Linear(512, 256),   
            # nn.ReLU(),
            nn.Linear(256, 128),   
            # nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(128, 32),
            # nn.Dropout(0.2),
            nn.ReLU(),
            # nn.Linear(64, 32),
            # nn.ReLU(),
            nn.Linear(32, 1),
        )


    def forward(self, x):
        x = self.l1(x)
        return x

In [16]:
def run_on_validation(val_loader, model):
    model.eval()
    val_preds = []
    val_labels = []
    with torch.no_grad():
        # for inputs, labels in tqdm(val_loader, desc=f"Epoch {epoch + 1}/{num_epochs} - Validation"):
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            val_preds.extend(outputs.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())
            
    val_rmse = mean_squared_error(val_labels, val_preds, squared=False)
    return val_rmse


# Train MLP model
def train_mlp_model(train_loader, val_loader, model, criterion, optimizer, num_epochs=5, patience=10):
    model = model.to(device)


    best_val_rmse = float('inf')

    for epoch in tqdm(range(num_epochs)):
        model.train()
        # for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs} - Training"):
        train_preds = []
        train_labels = []
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            
            loss = criterion(outputs, labels.unsqueeze(1).float())
            loss.backward()
            optimizer.step()

        
            train_preds.extend(outputs.detach().cpu().numpy())
            train_labels.extend(labels.detach().cpu().numpy())

        train_rmse = mean_squared_error(train_labels, train_preds, squared=False)
        print(f"Epoch {epoch + 1}/{num_epochs}, Test RMSE: {train_rmse:.4f}")


        val_rmse = run_on_validation(val_loader, model)
        print(f"Epoch {epoch + 1}/{num_epochs}, Validation RMSE: {val_rmse:.4f}")

        if val_rmse < best_val_rmse:
            best_val_rmse = val_rmse
            no_improvement = 0
        else:
            no_improvement += 1
            if no_improvement >= patience:
                print(f"No improvement for {patience} consecutive epochs. Early stopping.")
                break


In [17]:
# from transformers import AutoTokenizer, AutoModel
# import torch
# import numpy as np

# def get_embeddings(texts, model_name):
#     # Load pre-trained model and tokenizer
#     tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=514)
#     model = AutoModel.from_pretrained(
#         model_name,
#     ).to(device)

#     embeddings = []

#     for text in tqdm(texts):
#         # Tokenize input text and obtain embeddings
#         inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
#         with torch.no_grad():
#             outputs = model(**inputs)
#         # Extract the embeddings from the last layer
#         last_hidden_states = outputs.last_hidden_state
#         # Average pooling to get a fixed-size embedding for the entire text
#         avg_pooling = torch.mean(last_hidden_states, dim=1)
#         # Convert tensor to numpy array
#         avg_pooling = avg_pooling.cpu().numpy()
#         # Append the embedding to the list
#         embeddings.append(avg_pooling)

#     return np.concatenate(embeddings, axis=0)

# models = [
#     'xlm-mlm-enfr-1024',
#     'distilbert-base-cased',
#     'bert-base-uncased',
#     'roberta-base',
    
    
#     'bert-base-cased',
#     'xlm-roberta-base',
#     'xlnet-base-cased',    

# # 'cardiffnlp/twitter-roberta-base-sentiment',
# #     'ctrl',
# #     'transfo-xl-wt103',
# #     'openai-gpt',
# #     'gpt2'
# ]


# def get_all_embeddings(texts):
#     all_embeddings = []

#     for model_name in models:
#         print(f"calculating embeddings for {model_name}")
#         embeddings = get_embeddings(texts, model_name)
#         all_embeddings.append(embeddings)

#     return np.concatenate(all_embeddings, axis=1)




In [55]:
# Tokenize and create BERT embeddings
def create_gpt2_embeddings(texts):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    tokenizer.pad_token = tokenizer.eos_token
    model = GPT2Model.from_pretrained('gpt2')
    model = model.to(device)

    embeddings = []
    for text in tqdm(texts, desc="Creating GPT-2 Embeddings"):
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
        inputs = inputs.to(device)
        
        with torch.no_grad():
            outputs = model(**inputs)
            last_hidden_states = outputs.last_hidden_state

        # Use the mean of the last hidden states as the embedding
        embedding = torch.mean(last_hidden_states, dim=1).squeeze().detach().cpu().numpy()
        embeddings.append(embedding)

    return embeddings



def create_bert_embeddings(texts):
    bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    bert_model = BertModel.from_pretrained(
        'bert-base-uncased',
        output_hidden_states = True,
    ).to(device)

    
    embeddings = []
    num_layers = 13
    max_length = 512
    for text in tqdm(texts, desc="Creating BERT Embeddings"):
        head = bert_tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(device)
        
        ind = max(len(text.split(' ')) - 2 * max_length, 0)
        tail = bert_tokenizer(text[ind:], return_tensors='pt', truncation=True, padding=True).to(device)

        with torch.no_grad():
            output_head = bert_model(**head)
            output_tail = bert_model(**tail)
        
            
        embedding_head = torch.cat([output_head.hidden_states[i] for i in range(num_layers)], dim=-1)
        embedding_tail = torch.cat([output_tail.hidden_states[i] for i in range(num_layers)], dim=-1)    
        
        mean_embedding = (torch.mean(embedding_head, dim=1) + torch.mean(embedding_tail, dim=1)) / 2
        # mean_embedding = (torch.mean(embedding_head, dim=1)
                          
        final_embedding = mean_embedding.squeeze().detach().cpu().numpy()
        embeddings.append(final_embedding)

    return embeddings
    

    # max_length=512
    # num_layers = 13
    # embeddings = []
    # counter = 0
    # for text in tqdm(texts, desc="Creating BERT Embeddings"):
    #     counter += 1
    #     if counter >= 100:
    #         gc.collect()
    #         counter = 0
    #         libc = ctypes.CDLL("libc.so.6") # clearing cache 
    #         libc.malloc_trim(0)
        
    #     inputs = bert_tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(device)
    #     input_ids = inputs['input_ids']
    #     attention_mask = inputs['attention_mask']

    #     # Determine the length of the text
    #     text_length = len(input_ids[0])

    #     # Determine the length of the head and tail for each text
    #     head_length = min(max_length, text_length)
    #     # tail_length = max_length - head_length
    #     tail_length = head_length

    #     # Extract the head and tail embeddings
    #     head_inputs = {'input_ids': input_ids[:, :head_length], 'attention_mask': attention_mask[:, :head_length]}
    #     tail_inputs = {'input_ids': input_ids[:, -tail_length:], 'attention_mask': attention_mask[:, -tail_length:]}

    #     # head_inputs = {key: value.to(device) for key, value in head_inputs.items()}
    #     # tail_inputs = {key: value.to(device) for key, value in tail_inputs.items()}

    #     assert head_inputs['input_ids'].size()[1] == head_length
    #     assert tail_inputs['input_ids'].size()[1] == head_length
    #     assert head_inputs['attention_mask'].size()[1] == head_length
    #     assert tail_inputs['attention_mask'].size()[1] == head_length


    #     with torch.no_grad():
    #         head_outputs = bert_model(**head_inputs)
    #         tail_outputs = bert_model(**tail_inputs)

    #         # head_hidden_states = torch.stack(head_outputs.hidden_states[-num_layers:])
    #         # tail_hidden_states = torch.stack(tail_outputs.hidden_states[-num_layers:])
            
    #         h_e = torch.cat([head_outputs.hidden_states[i] for i in range(num_layers)], dim=-1)
    #         t_e = torch.cat([tail_outputs.hidden_states[i] for i in range(num_layers)], dim=-1)

                
    #         # Average the embeddings for the head and tail
    #         final_embedding = (torch.mean(h_e, dim=1) + torch.mean(t_e, dim=1)) / 2

    #         # print(head_outputs.hidden_states[1].size())
    #         # print(head_hidden_states.size())
    #         print(h_e.size())
    #         # print(final_embedding.size())


    
    #     # del head_hidden_states
    #     # del tail_hidden_states
    #     # del head_outputs
    #     # del tail_outputs
    #     # del head_inputs
    #     # del tail_inputs
    #     # del inputs

    #     final_embedding = final_embedding.squeeze().detach().cpu().numpy()
    #     embeddings.append(final_embedding)

    # return embeddings


In [56]:
def replace_keywords(text):
    words = text.split()

    for i in range(len(words)):
        if words[i].startswith('@ORGANIZATION'):
            words[i] = 'organization'
        elif words[i].startswith('@TIME'):
            words[i] = '2PM'
        elif words[i].startswith('@NUM'):
            words[i] = '5'
        elif words[i].startswith('@MONEY'):
            words[i] = 'dollar'
        elif words[i].startswith('@PERCENT'):
            words[i] = '50%'
        elif words[i].startswith('@LOCATION'):
            words[i] = 'Waterloo'
        elif words[i].startswith('@DATE'):
            words[i] = 'November 17'
        elif words[i].startswith('@CITY'):
            words[i] = 'Waterloo'

    return ' '.join(words)


def replace_keyword_list(texts):
    return list(map(replace_keywords, texts))


def embedding_after_replacement(texts):
    return create_bert_embeddings(replace_keyword_list(texts))




In [57]:
train_texts, train_scores = load_data('train.csv')
train_texts, val_texts, train_scores, val_scores = train_test_split(
    train_texts, train_scores, test_size=0.2, random_state=42
)

embedding_function = embedding_after_replacement
train_embeddings = embedding_function(train_texts)
val_embeddings = embedding_function(val_texts)

train_data = TensorDataset(torch.Tensor(train_embeddings), torch.tensor(train_scores))
val_data = TensorDataset(torch.Tensor(val_embeddings), torch.tensor(val_scores))


Creating BERT Embeddings: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 600/600 [00:22<00:00, 26.55it/s]


In [58]:
# Create DataLoader
patience = 5
batch_size = 256
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

model = MLPModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=20e-6)

num_epochs = 150
train_mlp_model(train_loader, val_loader, model, criterion, optimizer, num_epochs=num_epochs, patience=patience)

  1%|█▌                                                                                                                       | 2/150 [00:00<00:27,  5.47it/s]

Epoch 1/150, Test RMSE: 3.4192
Epoch 1/150, Validation RMSE: 3.3077
Epoch 2/150, Test RMSE: 3.1845
Epoch 2/150, Validation RMSE: 2.9113


  3%|███▏                                                                                                                     | 4/150 [00:00<00:26,  5.51it/s]

Epoch 3/150, Test RMSE: 2.5843
Epoch 3/150, Validation RMSE: 1.9717
Epoch 4/150, Test RMSE: 1.4387
Epoch 4/150, Validation RMSE: 0.6794


  4%|████▊                                                                                                                    | 6/150 [00:01<00:26,  5.52it/s]

...

Epoch 77/150, Test RMSE: 0.3258
Epoch 77/150, Validation RMSE: 0.3393
Epoch 78/150, Test RMSE: 0.3249


 53%|███████████████████████████████████████████████████████████████▏                                                        | 79/150 [00:14<00:13,  5.44it/s]

Epoch 78/150, Validation RMSE: 0.3393
Epoch 79/150, Test RMSE: 0.3243
Epoch 79/150, Validation RMSE: 0.3406


 53%|███████████████████████████████████████████████████████████████▏                                                        | 79/150 [00:14<00:13,  5.45it/s]

Epoch 80/150, Test RMSE: 0.3236
Epoch 80/150, Validation RMSE: 0.3387
No improvement for 5 consecutive epochs. Early stopping.


In [59]:
test_texts, _ = load_data('sample_submission.csv')
test_embeddings = embedding_function(test_texts)


test_data = TensorDataset(torch.tensor(test_embeddings))
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)



model.eval()
test_preds = []
with torch.no_grad():
    for inputs in tqdm(test_loader, desc="Generating Test Predictions"):
        inputs = inputs[0].to(device)

        outputs = model(inputs)
        test_preds.extend(outputs.cpu().numpy())


test_preds = [x[0] for x  in test_preds]



now = datetime.datetime.now().strftime("%b_%d_%H:%M")
validation_score = run_on_validation(val_loader, model)
model_name = f"bert-base-uncased_13layers_early_stopping_head_1.5backtail_{now}_{validation_score}.csv"

print(model_name)
        
# Write predictions to the output CSV file
output_df = pd.DataFrame({'text': test_texts, 'score': test_preds})
output_df.to_csv(model_name, index=False)


Generating Test Predictions: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 167.51it/s]

bert-base-uncased_13layers_early_stopping_head_1.5backtail_Nov_17_20:42_0.3386967182159424.csv
